In [15]:
%pip install -q pydantic aiohttp

Note: you may need to restart the kernel to use updated packages.


In [34]:
import aiohttp
import asyncio
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from datetime import datetime

class Customer(BaseModel):
    name: str
    id: int

class State(BaseModel):
    name: str
    id: int

class Region(BaseModel):
    tree_path_id: str = Field(..., alias='treePathId')
    socr: str
    id: int
    oktmo: str
    code: str
    name: str

class File(BaseModel):
    company_id: Optional[int] = Field(None, alias='companyId')
    name: str
    id: int
    content: Optional[bytes] = None

class DeliveryItem(BaseModel):
    sum: float
    cost_per_unit: float = Field(..., alias='costPerUnit')
    quantity: float
    name: str
    buyer_id: Optional[int] = Field(None, alias='buyerId')
    is_buyer_invitation_sent: bool = Field(..., alias='isBuyerInvitationSent')
    is_approved_by_buyer: Optional[bool] = Field(None, alias='isApprovedByBuyer')

class Delivery(BaseModel):
    period_days_from: Optional[int] = Field(None, alias='periodDaysFrom')
    period_days_to: Optional[int] = Field(None, alias='periodDaysTo')
    period_date_from: Optional[str] = Field(None, alias='periodDateFrom')
    period_date_to: Optional[str] = Field(None, alias='periodDateTo')
    delivery_place: str = Field(..., alias='deliveryPlace')
    quantity: float
    items: List[DeliveryItem]
    id: int

class AuctionItem(BaseModel):
    current_value: float = Field(..., alias='currentValue')
    cost_per_unit: float = Field(..., alias='costPerUnit')
    okei_name: str = Field(..., alias='okeiName')
    created_offer_id: Optional[int] = Field(None, alias='createdOfferId')
    sku_id: Optional[int] = Field(None, alias='skuId')
    image_id: Optional[int] = Field(None, alias='imageId')
    default_image_id: Optional[int] = Field(None, alias='defaultImageId')
    okpd_name: str = Field(..., alias='okpdName')
    production_directory_name: str = Field(..., alias='productionDirectoryName')
    oksm: Optional[str]
    name: Optional[str]
    id: int

class Bet(BaseModel):
    num: int
    cost: float
    server_time: str = Field(..., alias='serverTime')
    is_auto_bet: bool = Field(..., alias='isAutoBet')
    auction_id: int = Field(..., alias='auctionId')
    supplier_id: int = Field(..., alias='supplierId')
    create_user_id: int = Field(..., alias='createUserId')
    last_manual_server_time: Optional[str] = Field(None, alias='lastManualServerTime')
    id: int

class TenderData(BaseModel):
    customer: Customer
    created_by_customer: Customer = Field(..., alias='createdByCustomer')
    state: State
    start_date: str = Field(..., alias='startDate')
    initial_duration: float = Field(..., alias='initialDuration')
    end_date: str = Field(..., alias='endDate')
    start_cost: float = Field(..., alias='startCost')
    next_cost: Optional[float] = Field(None, alias='nextCost')
    last_bet_cost: Optional[float] = Field(None, alias='lastBetCost')
    step: float
    auction_item: List[AuctionItem] = Field(..., alias='auctionItem')
    bets: List[Bet]
    unique_supplier_count: int = Field(..., alias='uniqueSupplierCount')
    auction_region: List[Region] = Field(..., alias='auctionRegion')
    repeat_id: Optional[int] = Field(None, alias='repeatId')
    unpublish_name: Optional[str] = Field(None, alias='unpublishName')
    unpublish_date: Optional[str] = Field(None, alias='unpublishDate')
    federal_law_name: str = Field(..., alias='federalLawName')
    conclusion_reason_name: Optional[str] = Field(None, alias='conclusionReasonName')
    items: List[AuctionItem]
    deliveries: List[Delivery]
    files: List[File]
    license_files: List[File] = Field(..., alias='licenseFiles')
    is_electronic_contract_execution_required: bool = Field(..., alias='isElectronicContractExecutionRequired')
    is_contract_guarantee_required: bool = Field(..., alias='isContractGuaranteeRequired')
    contract_guarantee_amount: Optional[float] = Field(None, alias='contractGuaranteeAmount')
    is_license_production: bool = Field(..., alias='isLicenseProduction')
    name: str
    id: int

class TenderParser:
    def __init__(self):
        self.base_url = "https://zakupki.mos.ru/newapi/api/Auction/Get"
        self.file_url = "https://zakupki.mos.ru/newapi/api/FileStorage/Download"
        self.headers = {'accept': 'application/json'}

    async def fetch_tender(self, session: aiohttp.ClientSession, auction_id: int) -> Dict:
        params = {'auctionId': auction_id}
        async with session.get(self.base_url, params=params, headers=self.headers) as response:
            if response.status == 200:
                data = await response.json()
                return TenderData.model_validate(data)
            return None

    async def get_file_bytes(self, file_id: int) -> bytes:
        """Получение байтов файла по его ID"""
        async with aiohttp.ClientSession() as session:
            params = {'id': file_id}
            async with session.get(self.file_url, params=params) as response:
                if response.status == 200:
                    return await response.read()
                return None

    async def process_tenders(self, auction_ids: List[int], get_files: bool = False) -> List[TenderData]:
        """
        Обработка списка тендеров
        :param auction_ids: список ID тендеров
        :param get_files: флаг необходимости получения файлов
        :return: список тендеров
        """
        async with aiohttp.ClientSession() as session:
            tasks = [self.fetch_tender(session, aid) for aid in auction_ids]
            tenders = [r for r in await asyncio.gather(*tasks) if r is not None]
            
            if get_files:
                for tender in tenders:
                    for file in tender.files:
                        file.content = await self.get_file_bytes(file.id)
            
            return tenders

In [35]:
# Список ID тендеров для анализа
# https://zakupki.mos.ru/auction/9864533
# https://zakupki.mos.ru/auction/9864708
# https://zakupki.mos.ru/auction/9864771
# https://zakupki.mos.ru/auction/9864863
# https://zakupki.mos.ru/auction/9864870
# https://zakupki.mos.ru/auction/9864884
# https://zakupki.mos.ru/auction/9864977
# https://zakupki.mos.ru/auction/9862417
# https://zakupki.mos.ru/auction/9862374
# https://zakupki.mos.ru/auction/9862366

parser = TenderParser()
tenders = await parser.process_tenders([9864533, 9864708, 9864771, 9864863, 9864870, 9864884, 9864977, 9862417, 9862374, 9862366], get_files=True)

for tender in tenders:
    for file in tender.files:
        if file.content:
            print(f"File {file.name} size: {len(file.content)} bytes")

File ТЗ № П-ПУиЗР-2024.docx size: 137156 bytes
File Проект контракта.pdf size: 102593 bytes
File Проект договора.pdf size: 98080 bytes
File ТЗ сварщики.docx size: 943002 bytes
File Проект договора.pdf size: 118951 bytes
File Техническое задание одежда.docx size: 33944 bytes
File Проект контракта.pdf size: 98910 bytes
File Техническое_задание_Мебель (2).docx size: 84716 bytes
File Расчет НМЦК (21).xlsx size: 79797 bytes
File Техническое задание автозапчасти (4).docx size: 152731 bytes
File Проект контракта.pdf size: 103201 bytes
File Проект договора.pdf size: 98520 bytes
File ТЗ.docx size: 19633 bytes
File ТЗ вода.31.10.docx size: 16904 bytes
File Проект контракта.pdf size: 96910 bytes
File Проект контракта.pdf size: 98377 bytes
File ТЗ поставка диэл. СЗ.docx size: 44755 bytes
File Техническое задание сканер.doc size: 52224 bytes
File Проект контракта.pdf size: 95806 bytes
File техническое задание шпагат.docx size: 39960 bytes
File Проект контракта.pdf size: 95211 bytes
